In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
import sqlite3
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS
input_config = summary_data.INPUT_CONFIG

In [2]:
# Join parcel lookup data to parcel files to do some county and district summaries
# Load parcel geography lookup from soundcast db
conn = sqlite3.connect(r'../../../../inputs/db/soundcast_inputs_2023.db')
df_geog_lookup = pd.read_sql_query("SELECT ParcelID, CountyName, district_name, GrowthCenterName FROM parcel_2023_geography", conn)

# Land Use Summary
df_parcels = summary_data.load_landuse('landuse/parcels_urbansim.txt')
# Merge lookup data to parcels
df_parcels = df_parcels.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

# total parking cost
df_parcels['total_parking_cost'] = df_parcels['pprichrp'] * df_parcels['parkhr_p']

In [6]:
# input_config["model_year"]

In [7]:
df_parking_zones = pd.read_sql("SELECT * FROM parking_zones", con=conn)
df_parking_cost = pd.read_sql(
    "SELECT * FROM parking_costs WHERE year==" + input_config["model_year"], con=conn
)
df_parcels = pd.merge(
    left=df_parcels, right=df_parking_zones, left_on="taz_p", right_on="TAZ", how="left"
)

## Parking

### Hourly Parking Cost

- total hourly parking cost in parcel = `parkhr_p` * `pprichrp`

In [8]:
# # Daily parking cost totals (ppricdyp)
# pd.pivot_table(df_parcels, index='source', values='pprichrp', aggfunc='median').style.format('{:,.2f}')

# What percent of parcels have paid parking?
df_parcels['paid_parking'] = df_parcels['pprichrp'].apply(lambda x: 1 if x > 0 else 0)
# df_parcels[df_parcels['paid_parking']==1]
df_parcels['paid_parking'].value_counts(normalize=True).sort_index().to_frame().style.format('{:.2%}').set_caption('Percent of parcels with paid parking')

,paid_parking
0,99.97%
1,0.03%


In [9]:
# Parking costs by district
(pd.pivot_table(df_parcels, index='district_name', columns='source', values='paid_parking', aggfunc='sum')/\
    pd.pivot_table(df_parcels, index='district_name', columns='source', values='paid_parking', aggfunc='count')).style.format('{:,.1%}').set_caption('Paid parking by growth center')

source,current run
district_name,
East Side,0.0%
Everett-Lynwood-Edmonds,0.0%
Kitsap,0.0%
North Seattle-Shoreline,0.0%
Renton-FedWay-Kent,0.0%
S.Kitsap,0.0%
Seattle CBD,0.7%
South Pierce,0.0%
Suburban Snohomish,0.0%


In [15]:

# Sort ENS by the numeric value of the ENS column
df_parcels['ENS'] = df_parcels['ENS'].astype(str).str.extract('(\d+)').astype(int)
df_parcels['ENS'] = df_parcels['ENS'].astype(str).str.zfill(2)  

(pd.pivot_table(df_parcels, index='ENS', columns='source', values='paid_parking', aggfunc='sum')/\
    pd.pivot_table(df_parcels, index='ENS', columns='source', values='paid_parking', aggfunc='count')).style.format('{:,.1%}').set_caption('Paid parking by Parking Zone')\
    

source,current run
ENS,
00,0.0%
01,11.3%
02,0.4%
03,0.3%
04,2.4%
05,2.6%
06,21.1%
07,11.4%
08,8.7%


In [ ]:
(pd.pivot_table(df_parcels, index='district_name', columns='source', values='paid_parking', aggfunc='sum')/\
    pd.pivot_table(df_parcels, index='district_name', columns='source', values='paid_parking', aggfunc='count')).style.format('{:,.1%}').set_caption('Paid parking by growth center')\

In [30]:
(pd.pivot_table(df_parcels, index='GrowthCenterName', columns='source', values='paid_parking', aggfunc='sum')/\
    pd.pivot_table(df_parcels, index='GrowthCenterName', columns='source', values='paid_parking', aggfunc='count')).style.format('{:,.1%}').set_caption('Paid parking by growth center')\

source,current run
GrowthCenterName,
Auburn,0.0%
Bellevue,11.6%
Bothell Canyon Park,0.0%
Bremerton,1.3%
Burien,0.0%
Everett,0.5%
Federal Way,0.0%
Greater Downtown Kirkland,0.0%
Issaquah,0.0%


### Parking Supply

- The number of paid public off-street parking spaces with per-hour pricing

In [6]:
# Daily parking cost totals (ppricdyp)
pd.pivot_table(df_parcels, index='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

,parkhr_p
source,
current run,"202,147"


In [7]:
# Parking costs by district
pd.pivot_table(df_parcels, index='district_name', columns='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

source,current run
district_name,
East Side,"43,787"
Everett-Lynwood-Edmonds,"13,833"
Kitsap,"17,733"
North Seattle-Shoreline,"18,815"
Renton-FedWay-Kent,37
S.Kitsap,0
Seattle CBD,"80,205"
South Pierce,0
Suburban Snohomish,0


In [16]:
# Parking costs by zone
pd.pivot_table(df_parcels, index='ENS', columns='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

source,current run
ENS,
00,"9,967"
01,"42,374"
02,"10,766"
03,"12,484"
04,"21,601"
05,"18,732"
06,"25,905"
07,"10,444"
08,"8,145"


In [8]:
pd.pivot_table(df_parcels, index='GrowthCenterName', columns='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

source,current run
GrowthCenterName,
Auburn,0
Bellevue,"40,039"
Bothell Canyon Park,0
Bremerton,"9,686"
Burien,0
Everett,"13,641"
Federal Way,0
Greater Downtown Kirkland,0
Issaquah,0
